In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import pickle
import random
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2, 3, 4"
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
# logging.basicConfig(level=logging.INFO, filename='log_Model', format=LOG_FORMAT)
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)

from IPython.display import display, HTML
def show_df(df):
    display(HTML(df.to_html()))

# Read

In [2]:
logging.info('start read data')


pickle_path = '../user_data/data_sort_out/testB_dataset_mp.pickle'
with open(pickle_path, 'rb') as f:
    testB_dataset_mp = pickle.load(f)
key_list_testB, mp_action_testB, mp_distance_testB, mp_order_testB, mp_distance_testB_detail = \
testB_dataset_mp['key_list'], testB_dataset_mp['mp_action'], testB_dataset_mp['mp_distance'], testB_dataset_mp['mp_order'], testB_dataset_mp['mp_distance_detail']

pickle_path = '../user_data/data_sort_out/testB_dataset_df.pickle'
with open(pickle_path, 'rb') as f:
    testB_dataset_df = pickle.load(f)
df_actions_testB, df_couriers_testB, df_distances_testB, df_orders_testB = \
testB_dataset_df['df_actions'], testB_dataset_df['df_couriers'], testB_dataset_df['df_distances'], testB_dataset_df['df_orders']


pickle_path = '../user_data/data_sort_out/train_dataset_mp.pickle'
with open(pickle_path, 'rb') as f:
    train_dataset_mp = pickle.load(f)
key_list_train, mp_action_train, mp_distance_train, mp_order_train, mp_distance_train_detail = \
train_dataset_mp['key_list'], train_dataset_mp['mp_action'], train_dataset_mp['mp_distance'], train_dataset_mp['mp_order'], train_dataset_mp['mp_distance_detail']

pickle_path = '../user_data/data_sort_out/testA_dataset_mp.pickle'
with open(pickle_path, 'rb') as f:
    testA_dataset_mp = pickle.load(f)
key_list_testA, mp_action_testA, mp_distance_testA, mp_order_testA, mp_distance_testA_detail = \
testA_dataset_mp['key_list'], testA_dataset_mp['mp_action'], testA_dataset_mp['mp_distance'], testA_dataset_mp['mp_order'], testA_dataset_mp['mp_distance_detail']

pickle_path = '../user_data/data_sort_out/train_dataset_df.pickle'
with open(pickle_path, 'rb') as f:
    train_dataset_df = pickle.load(f)
df_actions_train, df_couriers_train, df_distances_train, df_orders_train = \
train_dataset_df['df_actions'], train_dataset_df['df_couriers'], train_dataset_df['df_distances'], train_dataset_df['df_orders']
    
pickle_path = '../user_data/data_sort_out/testA_dataset_df.pickle'
with open(pickle_path, 'rb') as f:
    testA_dataset_df = pickle.load(f)
df_actions_testA, df_couriers_testA, df_distances_testA, df_orders_testA = \
testA_dataset_df['df_actions'], testA_dataset_df['df_couriers'], testA_dataset_df['df_distances'], testA_dataset_df['df_orders']

pickle_path = '../user_data/data_sort_out/one_hot_name.pickle'
with open(pickle_path, 'rb') as f:
    one_hot_name = pickle.load(f)

logging.info('finish read data')

2020-04-13 21:43:49,512 - INFO - start read data
2020-04-13 22:10:03,722 - INFO - finish read data


In [4]:
pickle_path = '../user_data/build_dataset/df_train_info.pickle'
with open(pickle_path, 'rb') as f:
    df_train_info = pickle.load(f)

pickle_path = '../user_data/build_dataset/df_testA_info.pickle'
with open(pickle_path, 'rb') as f:
    df_testA_info = pickle.load(f)

pickle_path = '../user_data/build_dataset/df_testB_info.pickle'
with open(pickle_path, 'rb') as f:
    df_testB_info = pickle.load(f)


# build

In [5]:
delay_feature_list = []
logging.info('start delay feature build')

for i, (date, courier, wave_idx) in enumerate(key_list_train):
    if (i + 1) % 10000 == 0:
        logging.info('build %d train sample' % i)
        
    df_a_action = mp_action_train[date][courier][wave_idx]
    df_a_order = mp_order_train[date][courier][wave_idx]
    know_len = df_train_info.iloc[i].know_lens
    cnt_pickup_delay, cnt_deliver_delay = 0, 0
    delay_feature_dict = {}
    
    for j in range(know_len):
        action_row = df_a_action.iloc[j]
        tracking_id = action_row.tracking_id
        cur_time = action_row.expect_time
        df_order_info = df_a_order.query('tracking_id == @tracking_id')
        
        estimate_pick_time = df_order_info.estimate_pick_time
        promise_deliver_time = df_order_info.promise_deliver_time
        delay_feature_dict['courier_id'] = courier

        if action_row.action_type == 'PICKUP':
            delay_feature_dict['delay_type'] = 'PICKUP'
            delay_feature_dict['delay'] = (cur_time > estimate_pick_time).iloc[0]
            delay_feature_dict['delay_time'] = (cur_time - estimate_pick_time).iloc[0]
        else:
            delay_feature_dict['delay_type'] = 'DELIVERY'
            delay_feature_dict['delay'] = (cur_time > promise_deliver_time).iloc[0]
            delay_feature_dict['delay_time'] = (cur_time - promise_deliver_time).iloc[0]
        delay_feature_list.append(delay_feature_dict)

for i, (date, courier, wave_idx) in enumerate(key_list_testA):
    if (i + 1) % 1000 == 0:
        logging.info('build %d testA sample' % i)
        
    df_a_action = mp_action_testA[date][courier][wave_idx]
    df_a_order = mp_order_testA[date][courier][wave_idx]
    know_len = df_testA_info.iloc[i].know_lens
    cnt_pickup_delay, cnt_deliver_delay = 0, 0
    delay_feature_dict = {}
    
    for j in range(know_len):
        action_row = df_a_action.iloc[j]
        tracking_id = action_row.tracking_id
        cur_time = action_row.expect_time
        df_order_info = df_a_order.query('tracking_id == @tracking_id')
        
        estimate_pick_time = df_order_info.estimate_pick_time
        promise_deliver_time = df_order_info.promise_deliver_time
        delay_feature_dict['courier_id'] = courier
        
        if action_row.action_type == 'PICKUP':
            delay_feature_dict['delay_type'] = 'PICKUP'
            delay_feature_dict['delay'] = (cur_time > estimate_pick_time).iloc[0]
            delay_feature_dict['delay_time'] = (cur_time - estimate_pick_time).iloc[0]
        else:
            delay_feature_dict['delay_type'] = 'DELIVERY'
            delay_feature_dict['delay'] = (cur_time > promise_deliver_time).iloc[0]
            delay_feature_dict['delay_time'] = (cur_time - promise_deliver_time).iloc[0]
        
        delay_feature_list.append(delay_feature_dict)

for i, (date, courier, wave_idx) in enumerate(key_list_testB):
    if (i + 1) % 1000 == 0:
        logging.info('build %d testB sample' % i)
        
    df_a_action = mp_action_testB[date][courier][wave_idx]
    df_a_order = mp_order_testB[date][courier][wave_idx]
    know_len = df_testB_info.iloc[i].know_lens
    cnt_pickup_delay, cnt_deliver_delay = 0, 0
    delay_feature_dict = {}
    
    for j in range(know_len):
        action_row = df_a_action.iloc[j]
        tracking_id = action_row.tracking_id
        cur_time = action_row.expect_time
        df_order_info = df_a_order.query('tracking_id == @tracking_id')
        
        estimate_pick_time = df_order_info.estimate_pick_time
        promise_deliver_time = df_order_info.promise_deliver_time
        delay_feature_dict['courier_id'] = courier
        
        if action_row.action_type == 'PICKUP':
            delay_feature_dict['delay_type'] = 'PICKUP'
            delay_feature_dict['delay'] = (cur_time > estimate_pick_time).iloc[0]
            delay_feature_dict['delay_time'] = (cur_time - estimate_pick_time).iloc[0]
        else:
            delay_feature_dict['delay_type'] = 'DELIVERY'
            delay_feature_dict['delay'] = (cur_time > promise_deliver_time).iloc[0]
            delay_feature_dict['delay_time'] = (cur_time - promise_deliver_time).iloc[0]
        
        delay_feature_list.append(delay_feature_dict)

logging.info('finish delay feature build')


def apply_delay(df):
    df_delivery = df.query('delay_type == "DELIVERY"')
    df_pickup = df.query('delay_type == "PICKUP"')
    feature_dict = {'delivery_num' : df_delivery.shape[0], 'pickup_num' : df_pickup.shape[0]}
    feature_dict['delivery_delay_count'] = sum(df_delivery['delay'])
    feature_dict['pickup_delay_count'] = sum(df_pickup['delay'])
    
    if feature_dict['pickup_num'] == 0:
        feature_dict['pickup_delay_time_avg'] = 0
        feature_dict['pickup_delay_rate'] = 0
    else:
        feature_dict['pickup_delay_time_avg'] = df_pickup['delay_time'].apply(lambda x : x if x > 0 else 0).mean()
        feature_dict['pickup_delay_rate'] = feature_dict['pickup_delay_count'] / feature_dict['pickup_num']
    
    if feature_dict['delivery_num'] == 0:
        feature_dict['delivery_delay_time_avg'] = 0
        feature_dict['delivery_delay_rate'] = 0
    else:
        feature_dict['delivery_delay_time_avg'] = df_delivery['delay_time'].apply(lambda x : x if x > 0 else 0).mean()
        feature_dict['delivery_delay_rate'] = feature_dict['delivery_delay_count'] / feature_dict['delivery_num']


    return pd.Series(feature_dict)

df_delay_info = pd.DataFrame(delay_feature_list)
df_courier_delay_info = df_delay_info.groupby('courier_id').apply(apply_delay)

courier_delay_features = ['pickup_delay_rate', 'delivery_delay_rate', 'pickup_delay_time_avg', 'delivery_delay_time_avg',
                         'delivery_delay_count', 'pickup_delay_count']

for delay_feature in courier_delay_features:
    df_couriers_train[delay_feature] = df_couriers_train['courier_id'].map(df_courier_delay_info[delay_feature])
    df_couriers_testA[delay_feature] = df_couriers_testA['courier_id'].map(df_courier_delay_info[delay_feature])  
    
    
def build_couriers_features(df_couriers, key_list):
    
    df_index_couriers = df_couriers.groupby('courier_id').apply(lambda x : x.groupby('date').apply(lambda x:x) )
    couriers_set = set(df_couriers['courier_id'])
    mp_couriers_features = {}
    for courier in couriers_set:
        df_a_couriers = df_index_couriers.loc[courier]
        mp_couriers_features[courier] = {}
        for idx in df_a_couriers.index:
            date = df_a_couriers.at[idx, 'date']
            mp_couriers_features[courier][date] = df_a_couriers.loc[idx]
    return mp_couriers_features

logging.info('start building couriers features')
mp_couriers_features_train = build_couriers_features(df_couriers_train, key_list_train)
mp_couriers_features_testA = build_couriers_features(df_couriers_testA, key_list_testA)
mp_couriers_features_testB = build_couriers_features(df_couriers_testB, key_list_testB)
logging.info('finish building couriers features')


2020-04-13 22:11:06,589 - INFO - start delay feature build
2020-04-13 22:12:34,928 - INFO - build 9999 train sample
2020-04-13 22:14:17,616 - INFO - build 19999 train sample
2020-04-13 22:15:59,771 - INFO - build 29999 train sample
2020-04-13 22:18:29,566 - INFO - build 39999 train sample
2020-04-13 22:20:14,201 - INFO - build 49999 train sample
2020-04-13 22:21:59,781 - INFO - build 59999 train sample
2020-04-13 22:25:13,842 - INFO - build 69999 train sample
2020-04-13 22:29:39,724 - INFO - build 79999 train sample
2020-04-13 22:30:53,800 - INFO - build 999 testA sample
2020-04-13 22:31:15,514 - INFO - build 1999 testA sample
2020-04-13 22:31:37,101 - INFO - build 2999 testA sample
2020-04-13 22:31:59,560 - INFO - build 3999 testA sample
2020-04-13 22:32:17,971 - INFO - build 999 testB sample
2020-04-13 22:32:30,617 - INFO - build 1999 testB sample
2020-04-13 22:32:43,482 - INFO - build 2999 testB sample
2020-04-13 22:32:56,610 - INFO - build 3999 testB sample
2020-04-13 22:32:59,078 

In [6]:
pickle_path = '../user_data/generate_train_test_courier_feature/mp_couriers_features_train.pickle'
with open(pickle_path, 'wb') as f:
    pickle.dump(mp_couriers_features_train, f)

pickle_path = '../user_data/generate_train_test_courier_feature/mp_couriers_features_testA.pickle'
with open(pickle_path, 'wb') as f:
    pickle.dump(mp_couriers_features_testA, f)
    
pickle_path = '../user_data/generate_train_test_courier_feature/mp_couriers_features_testB.pickle'
with open(pickle_path, 'wb') as f:
    pickle.dump(mp_couriers_features_testB, f)